In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import seaborn as sns

from os.path import join

plt.style.use(["seaborn", "thesis"])

/home/jo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
plt.rc("figure", figsize=(8,4))

# Fetch Dataset 

In [3]:
from SCFInitialGuess.utilities.dataset import extract_triu_batch, AbstractDataset
from sklearn.model_selection import train_test_split

data_path = "../../dataset/TSmall_sto3g"
postfix = "TSmall_sto3g"
dim = 26
data_path = "../../dataset/EthenT/"
postfix = "EthenT"
dim = 72
basis = "6-311++g**"
n_electrons = 16
#data_path = "../butadien/data/"
#postfix = ""
#dim = 26


def split(x, y, ind):
    return x[:ind], y[:ind], x[ind:], y[ind:]

S = np.load(join(data_path, "S" + postfix + ".npy"))
P = np.load(join(data_path, "P" + postfix + ".npy"))

index = np.load(join(data_path, "index" + postfix + ".npy"))

molecules = np.load(join(data_path, "molecules" + postfix + ".npy"))


ind = int(0.8 * len(index))
ind_val = int(0.8 * ind)


molecules = (
    molecules[:ind_val], 
    molecules[ind_val:ind], 
    molecules[ind:]
)

s_triu_norm, mu, std = AbstractDataset.normalize(S)


s_train, p_train, s_test, p_test = split(S, P, ind)
s_train, p_train, s_val, p_val = split(s_train, p_train, ind_val)



In [4]:
from SCFInitialGuess.utilities.dataset import make_center_block_dataset
from SCFInitialGuess.descriptors.high_level import AtomicNumberWeighted
from SCFInitialGuess.descriptors.cutoffs import BehlerCutoff1
from SCFInitialGuess.descriptors.models import RADIAL_GAUSSIAN_MODELS
from SCFInitialGuess.descriptors.coordinate_descriptors import \
    Gaussians, SPHAngularDescriptor

descriptor = AtomicNumberWeighted(
    Gaussians(*RADIAL_GAUSSIAN_MODELS["Man"]),
    SPHAngularDescriptor(15),
    BehlerCutoff1(5)
)
    
    
dataset = make_center_block_dataset(
    descriptor,
    molecules,
    (p_train, p_val, p_test),
    "C"
)

[[ 6.57554709e-01  5.63376824e-01 -8.87032655e-02 ...  2.19218274e-04
   2.41665076e-04  3.12425062e-04]
 [ 6.59757577e-01  5.67672344e-01 -1.03730826e-01 ... -3.25049742e-05
  -5.57079416e-06  2.43427427e-05]
 [ 6.59942694e-01  5.68118906e-01 -1.03002182e-01 ... -3.53700573e-05
   6.31675933e-08  6.56060703e-05]
 ...
 [ 6.57599232e-01  5.63017768e-01 -9.33231518e-02 ... -3.28731017e-04
   3.81563628e-04  1.35645019e-03]
 [ 6.57362681e-01  5.63198171e-01 -8.90230086e-02 ... -1.51238683e-04
   3.47754865e-04  1.67687644e-03]
 [ 6.58106099e-01  5.63782457e-01 -9.82513394e-02 ...  2.63695244e-05
  -6.61126928e-05  3.88821027e-04]] [<SCFInitialGuess.utilities.dataset.Molecule object at 0x7f3d02161588>
201 201


[[ 6.57424294e-01  5.62670650e-01 -9.36513099e-02 ... -2.80544024e-04
   8.30003337e-05  2.07562427e-04]
 [ 6.57582752e-01  5.64294889e-01 -8.59821475e-02 ...  1.64461712e-04
  -1.06567935e-03  1.82424837e-03]
 [ 6.57437724e-01  5.62903593e-01 -9.29789139e-02 ... -8.46575291e-05
  -1.99977274e-06  2.03690504e-04]
 ...
 [ 6.57636711e-01  5.62699688e-01 -9.56375269e-02 ... -3.38324808e-04
  -3.11115856e-04  1.09414796e-03]
 [ 6.57638221e-01  5.63575238e-01 -9.38642435e-02 ... -1.88887843e-04
  -5.66089764e-04  1.78422644e-03]
 [ 6.58319308e-01  5.64722796e-01 -9.48165752e-02 ... -1.10560207e-04
  -2.15595829e-04  2.12911723e-03]] [<SCFInitialGuess.utilities.dataset.Molecule object at 0x7f3d0219f208>
160 160


[[ 6.57217377e-01  5.62207858e-01 -9.29441147e-02 ...  2.28190091e-04
  -2.74425395e-04  4.23264573e-04]
 [ 6.60775634e-01  5.69059210e-01 -1.09697488e-01 ...  4.55900803e-04
  -1.84870848e-04  7.71116743e-04]
 [ 6.58949196e-01  5.64872475e-01 -1.01404607e-01 ...  9.88760570e-05
  -2.01454047e-04  1.05658257e-03]
 ...
 [ 6.57823440e-01  5.63702217e-01 -9.25887236e-02 ... -3.23170671e-04
   3.77054003e-04  1.35537202e-03]
 [ 6.57582814e-01  5.62964127e-01 -9.27889331e-02 ... -2.92823907e-04
   3.91984329e-04  1.27016273e-03]
 [ 6.60261578e-01  5.67865509e-01 -1.09831405e-01 ... -3.12690310e-04
  -9.78873254e-05  1.15422430e-04]] [<SCFInitialGuess.utilities.dataset.Molecule object at 0x7f3d07fc9588>
640 640


# Utilities 

## Calculate Descriptors and extract center blocks 

## Networks 

In [19]:
#keras.backend.clear_session()

#activation = "elu"
#learning_rate = 1e-5

intializer = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01)

def make_model(
        structure, 
        input_dim, 
        output_dim,
        activation="elu", 
        learning_rate=1e-3
    ):

    model = keras.Sequential()

    # input layer
    model.add(keras.layers.Dense(
        structure[0], 
        activation=activation, 
        input_dim=input_dim, 
        kernel_initializer=intializer
    ))

    for layer in structure[1:]:
        model.add(keras.layers.Dense(
            layer, 
            activation=activation, 
            #kernel_initializer=intializer, 
            #bias_initializer='zeros',
            kernel_regularizer=keras.regularizers.l2(1e-6)
        ))

    #output
    model.add(keras.layers.Dense(output_dim))

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate), 
        loss='MSE', 
        metrics=['mae', 'mse']
    )
    
    return model



In [20]:
filepath = "../../models/ParticleNumberIndependent/CenterBlocks/model_C_" + postfix + ".h5"

early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_mean_squared_error", 
    min_delta=1e-10, 
    patience=200, 
    verbose=1
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_mean_squared_error', 
    factor=0.5, 
    patience=50, 
    verbose=1, 
    mode='auto', 
    min_delta=1e-6, 
    cooldown=50, 
    min_lr=1e-10
)

checkpoint = keras.callbacks.ModelCheckpoint(
    filepath, 
    monitor='val_mean_squared_error', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', 
    period=1
)

epochs = 1000


def train_model(model, dataset, learning_rate=1e-4):
    
    error = []
    while True:
        keras.backend.set_value(model.optimizer.lr, learning_rate)
            
        history = model.fit(
            x = dataset.training[0],
            y = dataset.training[1],
            epochs=epochs,
            shuffle=True,
            validation_data=dataset.validation, 
            verbose=1, 
            callbacks=[
                early_stopping, 
                reduce_lr,
                checkpoint
            ]
        )

        error.append(model.evaluate(
            dataset.testing[0], 
            dataset.testing[1], 
            verbose=1
        )[1])
    
    return error
    

# C  

## Compute inputs 

In [21]:
from SCFInitialGuess.utilities.constants import number_of_basis_functions as N_BASIS


species = "C"

dim_C = N_BASIS[basis][species]
dim_C_triu = dim_C * (dim_C + 1) // 2


In [22]:
structure = [dim_C_triu - 50, dim_C_triu + 50]

In [23]:
keras.backend.clear_session()
model = make_model(
    structure=structure,
    input_dim=descriptor.number_of_descriptors,
    output_dim=dim_C_triu
)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 203)               114289    
_________________________________________________________________
dense_1 (Dense)              (None, 303)               61812     
_________________________________________________________________
dense_2 (Dense)              (None, 253)               76912     
Total params: 253,013
Trainable params: 253,013
Non-trainable params: 0
_________________________________________________________________


In [ ]:
train_model(model, dataset, learning_rate=1e-3)

Train on 1280 samples, validate on 320 samples
Epoch 1/1000
1280/1280 [==============================] - 1s 461us/step - loss: 0.0129 - mean_absolute_error: 0.0764 - mean_squared_error: 0.0126 - val_loss: 0.0036 - val_mean_absolute_error: 0.0434 - val_mean_squared_error: 0.0034

Epoch 00001: val_mean_squared_error improved from inf to 0.00341, saving model to ../../models/ParticleNumberIndependent/CenterBlocks/model_C_EthenT.h5
Epoch 2/1000
1280/1280 [==============================] - 0s 153us/step - loss: 0.0019 - mean_absolute_error: 0.0301 - mean_squared_error: 0.0017 - val_loss: 0.0011 - val_mean_absolute_error: 0.0225 - val_mean_squared_error: 9.3021e-04

Epoch 00002: val_mean_squared_error improved from 0.00341 to 0.00093, saving model to ../../models/ParticleNumberIndependent/CenterBlocks/model_C_EthenT.h5
Epoch 3/1000
1280/1280 [==============================] - 0s 151us/step - loss: 0.0010 - mean_absolute_error: 0.0195 - mean_squared_error: 8.3593e-04 - val_loss: 8.4723e-04 - 


Epoch 00022: val_mean_squared_error improved from 0.00014 to 0.00014, saving model to ../../models/ParticleNumberIndependent/CenterBlocks/model_C_EthenT.h5
Epoch 23/1000
1280/1280 [==============================] - 0s 167us/step - loss: 2.3347e-04 - mean_absolute_error: 0.0060 - mean_squared_error: 8.9018e-05 - val_loss: 2.8356e-04 - val_mean_absolute_error: 0.0065 - val_mean_squared_error: 1.4060e-04

Epoch 00023: val_mean_squared_error did not improve from 0.00014
Epoch 24/1000
1280/1280 [==============================] - 0s 161us/step - loss: 2.2839e-04 - mean_absolute_error: 0.0058 - mean_squared_error: 8.6830e-05 - val_loss: 2.8725e-04 - val_mean_absolute_error: 0.0066 - val_mean_squared_error: 1.4717e-04

Epoch 00024: val_mean_squared_error did not improve from 0.00014
Epoch 25/1000
1280/1280 [==============================] - 0s 155us/step - loss: 2.2315e-04 - mean_absolute_error: 0.0058 - mean_squared_error: 8.4478e-05 - val_loss: 2.7125e-04 - val_mean_absolute_error: 0.0063 -

1280/1280 [==============================] - 0s 159us/step - loss: 1.4362e-04 - mean_absolute_error: 0.0047 - mean_squared_error: 6.3127e-05 - val_loss: 1.9173e-04 - val_mean_absolute_error: 0.0051 - val_mean_squared_error: 1.1252e-04

Epoch 00046: val_mean_squared_error improved from 0.00012 to 0.00011, saving model to ../../models/ParticleNumberIndependent/CenterBlocks/model_C_EthenT.h5
Epoch 47/1000
1280/1280 [==============================] - 0s 191us/step - loss: 1.3934e-04 - mean_absolute_error: 0.0046 - mean_squared_error: 6.1349e-05 - val_loss: 1.8956e-04 - val_mean_absolute_error: 0.0052 - val_mean_squared_error: 1.1282e-04

Epoch 00047: val_mean_squared_error did not improve from 0.00011
Epoch 48/1000
1280/1280 [==============================] - 0s 130us/step - loss: 1.3862e-04 - mean_absolute_error: 0.0046 - mean_squared_error: 6.3068e-05 - val_loss: 1.9048e-04 - val_mean_absolute_error: 0.0052 - val_mean_squared_error: 1.1618e-04

Epoch 00048: val_mean_squared_error did not

1280/1280 [==============================] - 0s 143us/step - loss: 8.9620e-05 - mean_absolute_error: 0.0044 - mean_squared_error: 5.7951e-05 - val_loss: 1.4563e-04 - val_mean_absolute_error: 0.0049 - val_mean_squared_error: 1.1466e-04

Epoch 00071: val_mean_squared_error did not improve from 0.00011
Epoch 72/1000
1280/1280 [==============================] - 0s 143us/step - loss: 8.6227e-05 - mean_absolute_error: 0.0044 - mean_squared_error: 5.5901e-05 - val_loss: 1.3981e-04 - val_mean_absolute_error: 0.0049 - val_mean_squared_error: 1.1017e-04

Epoch 00072: val_mean_squared_error did not improve from 0.00011
Epoch 73/1000
1280/1280 [==============================] - 0s 134us/step - loss: 8.4879e-05 - mean_absolute_error: 0.0043 - mean_squared_error: 5.5849e-05 - val_loss: 1.3502e-04 - val_mean_absolute_error: 0.0048 - val_mean_squared_error: 1.0663e-04

Epoch 00073: val_mean_squared_error did not improve from 0.00011
Epoch 74/1000
1280/1280 [==============================] - 0s 144us/s

1280/1280 [==============================] - 0s 145us/step - loss: 6.6258e-05 - mean_absolute_error: 0.0045 - mean_squared_error: 5.6163e-05 - val_loss: 1.1732e-04 - val_mean_absolute_error: 0.0050 - val_mean_squared_error: 1.0747e-04

Epoch 00096: val_mean_squared_error did not improve from 0.00010
Epoch 97/1000
1280/1280 [==============================] - 0s 130us/step - loss: 6.6044e-05 - mean_absolute_error: 0.0044 - mean_squared_error: 5.6422e-05 - val_loss: 1.2258e-04 - val_mean_absolute_error: 0.0050 - val_mean_squared_error: 1.1320e-04

Epoch 00097: val_mean_squared_error did not improve from 0.00010
Epoch 98/1000
1280/1280 [==============================] - 0s 145us/step - loss: 6.2899e-05 - mean_absolute_error: 0.0044 - mean_squared_error: 5.3710e-05 - val_loss: 1.1416e-04 - val_mean_absolute_error: 0.0049 - val_mean_squared_error: 1.0518e-04

Epoch 00098: val_mean_squared_error did not improve from 0.00010
Epoch 99/1000
1280/1280 [==============================] - 0s 131us/s

...


Epoch 00315: val_mean_squared_error did not improve from 0.00001
Epoch 316/1000
1280/1280 [==============================] - 0s 111us/step - loss: 1.4783e-06 - mean_absolute_error: 6.6262e-04 - mean_squared_error: 8.9915e-07 - val_loss: 1.4892e-05 - val_mean_absolute_error: 0.0012 - val_mean_squared_error: 1.4313e-05

Epoch 00316: val_mean_squared_error did not improve from 0.00001
Epoch 317/1000
1280/1280 [==============================] - 0s 99us/step - loss: 1.4670e-06 - mean_absolute_error: 6.5956e-04 - mean_squared_error: 8.8900e-07 - val_loss: 1.4609e-05 - val_mean_absolute_error: 0.0012 - val_mean_squared_error: 1.4032e-05

Epoch 00317: val_mean_squared_error improved from 0.00001 to 0.00001, saving model to ../../models/ParticleNumberIndependent/CenterBlocks/model_C_EthenT.h5
Epoch 318/1000
1280/1280 [==============================] - 0s 108us/step - loss: 1.4799e-06 - mean_absolute_error: 6.6275e-04 - mean_squared_error: 9.0332e-07 - val_loss: 1.5074e-05 - val_mean_absolute_e

1280/1280 [==============================] - 0s 115us/step - loss: 1.4582e-06 - mean_absolute_error: 6.6673e-04 - mean_squared_error: 9.0935e-07 - val_loss: 1.4554e-05 - val_mean_absolute_error: 0.0012 - val_mean_squared_error: 1.4007e-05

Epoch 00341: val_mean_squared_error improved from 0.00001 to 0.00001, saving model to ../../models/ParticleNumberIndependent/CenterBlocks/model_C_EthenT.h5
Epoch 342/1000
1280/1280 [==============================] - 0s 115us/step - loss: 1.4525e-06 - mean_absolute_error: 6.6687e-04 - mean_squared_error: 9.0556e-07 - val_loss: 1.4848e-05 - val_mean_absolute_error: 0.0012 - val_mean_squared_error: 1.4302e-05

Epoch 00342: val_mean_squared_error did not improve from 0.00001
Epoch 343/1000
1280/1280 [==============================] - 0s 120us/step - loss: 1.4447e-06 - mean_absolute_error: 6.6475e-04 - mean_squared_error: 8.9870e-07 - val_loss: 1.5081e-05 - val_mean_absolute_error: 0.0012 - val_mean_squared_error: 1.4535e-05

Epoch 00343: val_mean_squared

1280/1280 [==============================] - 0s 187us/step - loss: 1.4205e-06 - mean_absolute_error: 6.6235e-04 - mean_squared_error: 8.9706e-07 - val_loss: 1.4563e-05 - val_mean_absolute_error: 0.0012 - val_mean_squared_error: 1.4040e-05

Epoch 00366: val_mean_squared_error did not improve from 0.00001
Epoch 367/1000
1280/1280 [==============================] - 0s 191us/step - loss: 1.4265e-06 - mean_absolute_error: 6.6557e-04 - mean_squared_error: 9.0390e-07 - val_loss: 1.4678e-05 - val_mean_absolute_error: 0.0012 - val_mean_squared_error: 1.4155e-05

Epoch 00367: val_mean_squared_error did not improve from 0.00001
Epoch 368/1000
1280/1280 [==============================] - 0s 189us/step - loss: 1.4282e-06 - mean_absolute_error: 6.6600e-04 - mean_squared_error: 9.0683e-07 - val_loss: 1.4935e-05 - val_mean_absolute_error: 0.0012 - val_mean_squared_error: 1.4414e-05

Epoch 00368: val_mean_squared_error did not improve from 0.00001
Epoch 369/1000
1280/1280 [=============================


Epoch 00390: val_mean_squared_error did not improve from 0.00001
Epoch 391/1000
1280/1280 [==============================] - 0s 176us/step - loss: 1.2040e-06 - mean_absolute_error: 5.8746e-04 - mean_squared_error: 6.9999e-07 - val_loss: 1.4621e-05 - val_mean_absolute_error: 0.0011 - val_mean_squared_error: 1.4118e-05

Epoch 00391: val_mean_squared_error did not improve from 0.00001
Epoch 392/1000
1280/1280 [==============================] - 0s 159us/step - loss: 1.2050e-06 - mean_absolute_error: 5.8744e-04 - mean_squared_error: 7.0188e-07 - val_loss: 1.4675e-05 - val_mean_absolute_error: 0.0011 - val_mean_squared_error: 1.4172e-05

Epoch 00392: val_mean_squared_error did not improve from 0.00001
Epoch 393/1000
1280/1280 [==============================] - 0s 135us/step - loss: 1.2109e-06 - mean_absolute_error: 5.8930e-04 - mean_squared_error: 7.0835e-07 - val_loss: 1.4739e-05 - val_mean_absolute_error: 0.0011 - val_mean_squared_error: 1.4238e-05

Epoch 00393: val_mean_squared_error did

1280/1280 [==============================] - 0s 144us/step - loss: 1.2004e-06 - mean_absolute_error: 5.9185e-04 - mean_squared_error: 7.1287e-07 - val_loss: 1.4483e-05 - val_mean_absolute_error: 0.0011 - val_mean_squared_error: 1.3996e-05

Epoch 00416: val_mean_squared_error did not improve from 0.00001
Epoch 417/1000
1280/1280 [==============================] - 0s 150us/step - loss: 1.1951e-06 - mean_absolute_error: 5.9071e-04 - mean_squared_error: 7.0807e-07 - val_loss: 1.4634e-05 - val_mean_absolute_error: 0.0011 - val_mean_squared_error: 1.4147e-05

Epoch 00417: val_mean_squared_error did not improve from 0.00001
Epoch 418/1000
1280/1280 [==============================] - 0s 153us/step - loss: 1.1952e-06 - mean_absolute_error: 5.9117e-04 - mean_squared_error: 7.0898e-07 - val_loss: 1.4571e-05 - val_mean_absolute_error: 0.0011 - val_mean_squared_error: 1.4085e-05

Epoch 00418: val_mean_squared_error did not improve from 0.00001
Epoch 419/1000
1280/1280 [=============================

In [ ]:

keras.backend.clear_session()

try: 
    model_C = keras.models.load_model(filepath)
except OSError:
    print("Model not found at " + filepath)
    model_C = make_model(
        structure=structure_C,
        input_dim=dataset_C.training[0].shape[1],
        output_dim=dim_C_triu
    )
    
model_C.summary()